In [1]:
!git clone https://github.com/nikhilchatta/DDPM-CIFAR10-Clean.git ddpm-div2k

Cloning into 'ddpm-div2k'...
remote: Enumerating objects: 2610, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 2610 (delta 5), reused 5 (delta 1), pack-reused 2595 (from 1)
Receiving objects: 100% (2610/2610), 169.92 MiB | 57.46 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (2612/2612), done.
Filtering content: 100% (3/3), 381.08 MiB | 97.87 MiB/s, done.


In [2]:
cd ddpm-div2k/


/users/PFS0270/nikhilchatta/ddpm-div2k


In [5]:
!tree . -L 2


.
├── Checkpoints
│   └── final_model.pt
├── data
│   ├── DIV2K_train_HR
│   └── DIV2K_train_HR.zip
├── DenoisingDiffusionProbabilityModel-ddpm-
├── Diffusion
│   ├── Diffusion.py
│   ├── div2k_dataloader.py
│   ├── __init__.py
│   ├── Model.py
│   ├── __pycache__
│   ├── Train.py
│   └── Unet.py
├── DiffusionFreeGuidence
│   ├── DiffusionCondition.py
│   ├── __init__.py
│   ├── ModelCondition.py
│   └── TrainCondition.py
├── MainCondition.py
├── Main.py
├── Metrics.ipynb
├── __pycache__
│   └── Scheduler.cpython-39.pyc
├── README.md
└── Scheduler.py

8 directories, 18 files


In [46]:
!python Main.py


Using device: cpu
Training on device: cpu
/users/PFS0270/nikhilchatta/ddpm-div2k/Diffusion/div2k_dataloader.py:22: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  image = (torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
Epoch [1/5], Loss: 2.7971
Epoch [2/5], Loss: 0.6886
Epoch [3/5], Loss: 0.3190
Epoch [4/5], Loss: 0.1982
Epoch [5/5], Loss: 0.1340
✅ Model saved to: ./Checkpoints/final_model.pt


In [56]:
import torch
import os
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F

# === Your same UNet used in training ===
class UNet(nn.Module):
    def __init__(self, T, ch=128, ch_mult=[1, 2, 3, 4], attn=[2], num_res_blocks=2, dropout=0.1):
        super(UNet, self).__init__()
        self.T = T
        self.ch = ch

        self.time_embed = nn.Sequential(
            nn.Linear(T, ch),
            nn.ReLU(),
            nn.Linear(ch, ch * 2)
        )

        self.conv1 = nn.Conv2d(3, ch, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(ch, ch * 2, kernel_size=3, padding=1)
        self.deconv = nn.ConvTranspose2d(ch * 2, ch, kernel_size=3, padding=1)
        self.out_conv = nn.Conv2d(ch, 3, kernel_size=3, padding=1)
        self.act = nn.ReLU()

    def forward(self, x, t):
        t_embed = self._timestep_embedding(t, self.T)
        t_embed = self.time_embed(t_embed).unsqueeze(-1).unsqueeze(-1)
        x = self.act(self.conv1(x) + t_embed[:, :self.ch])
        x = self.act(self.conv2(x) + t_embed)
        x = self.act(self.deconv(x) + t_embed[:, :self.ch])
        x = self.out_conv(x)
        return x

    def _timestep_embedding(self, timesteps, dim):
        half = dim // 2
        emb = torch.exp(torch.arange(half, dtype=torch.float32, device=timesteps.device) * -torch.log(torch.tensor(10000.0)) / (half - 1))
        emb = timesteps.float().unsqueeze(1) * emb.unsqueeze(0)
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
        if dim % 2 == 1:
            emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
        return emb

# === Your same GaussianDiffusionSampler ===
from Diffusion.Diffusion import GaussianDiffusionSampler

def generate_one_batch(model_path, save_dir, batch_size=64, image_size=128, device="cpu"):
    os.makedirs(save_dir, exist_ok=True)

    model = UNet(
        T=1000,
        ch=128,
        ch_mult=[1, 2, 3, 4],
        attn=[2],
        num_res_blocks=2,
        dropout=0.15
    ).to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    diffusion = GaussianDiffusionSampler(model, beta_1=1e-4, beta_T=0.02, T=1000).to(device)

    x_T = torch.randn((batch_size, 3, image_size, image_size)).to(device)

    with torch.no_grad():
        sampled_imgs = diffusion(x_T)
        sampled_imgs = (sampled_imgs.clamp(-1, 1) + 1) / 2

    for i in range(batch_size):
        save_image(sampled_imgs[i], os.path.join(save_dir, f"sample_{i+1}.png"))

    print(f"✅ Generated {batch_size} images in: {save_dir}")


In [57]:
generate_one_batch(
    model_path="./Checkpoints/final_model.pt",
    save_dir="./SampledImgs/batch_1",
    batch_size=64,
    image_size=128,
    device="cpu"
)


/tmp/slurmtmp.760800/ipykernel_3742497/1297012076.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750


In [61]:
!pip3 install --upgrade --force-reinstall numpy scipy


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 122.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Not uninstalling numpy at /users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages, outside environment /apps/project/ondemand/app_jupyter/4.1.5
    Can't uninstall 'numpy'. No files were found to uninstall.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Not uninstalling scipy at /users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages, outside environment /apps/project/ondemand/app_jupyter/4.1.5

In [10]:
ls ./SampledImgs/batch_1/


sample_10.png  sample_22.png  sample_34.png  sample_46.png  sample_58.png
sample_11.png  sample_23.png  sample_35.png  sample_47.png  sample_59.png
sample_12.png  sample_24.png  sample_36.png  sample_48.png  sample_5.png
sample_13.png  sample_25.png  sample_37.png  sample_49.png  sample_60.png
sample_14.png  sample_26.png  sample_38.png  sample_4.png   sample_61.png
sample_15.png  sample_27.png  sample_39.png  sample_50.png  sample_62.png
sample_16.png  sample_28.png  sample_3.png   sample_51.png  sample_63.png
sample_17.png  sample_29.png  sample_40.png  sample_52.png  sample_64.png
sample_18.png  sample_2.png   sample_41.png  sample_53.png  sample_6.png
sample_19.png  sample_30.png  sample_42.png  sample_54.png  sample_7.png
sample_1.png   sample_31.png  sample_43.png  sample_55.png  sample_8.png
sample_20.png  sample_32.png  sample_44.png  sample_56.png  sample_9.png
sample_21.png  sample_33.png  sample_45.png  sample_57.png


In [9]:
ls


Checkpoints/                               DiffusionFreeGuidence/  __pycache__/
data/                                      MainCondition.py        README.md
DenoisingDiffusionProbabilityModel-ddpm-/  Main.py                 SampledImgs/
Diffusion/                                 Metrics.ipynb           Scheduler.py


In [8]:
cd ddpm-div2k/

/users/PFS0270/nikhilchatta/ddpm-div2k


In [13]:
import numpy as np
from PIL import Image

def calculate_psnr(img1, img2, max_pixel=1.0):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(max_pixel / np.sqrt(mse))

# Load generated image
gen_img = Image.open("./SampledImgs/batch_1/sample_1.png").convert("RGB")
gen_img = np.array(gen_img).astype(np.float32) / 255.0  # Normalize to [0,1]

# Load ground truth image (adjust path if needed)
gt_img = Image.open("./data/DIV2K_train_HR/0001.png").convert("RGB")
gt_img = gt_img.resize((128, 128))  # Resize to match generated image
gt_img = np.array(gt_img).astype(np.float32) / 255.0

# Compute PSNR
psnr_value = calculate_psnr(gen_img, gt_img)
print("PSNR:", psnr_value)


PSNR: 4.863290530561363


In [14]:
import os
import numpy as np
from PIL import Image

def calculate_psnr(img1, img2, max_pixel=1.0):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def load_image(path, resize=(128, 128)):
    img = Image.open(path).convert("RGB")
    img = img.resize(resize)
    img = np.array(img).astype(np.float32) / 255.0
    return img

generated_dir = "./SampledImgs/batch_1/"
groundtruth_dir = "./data/DIV2K_train_HR/"

all_psnr_rounds = []

for round_num in range(1, 21):
    psnr_scores = []
    for i in range(1, 65):
        gen_path = os.path.join(generated_dir, f"sample_{i}.png")
        gt_path = os.path.join(groundtruth_dir, f"{str(i).zfill(4)}.png")

        if not os.path.exists(gen_path) or not os.path.exists(gt_path):
            continue

        gen_img = load_image(gen_path)
        gt_img = load_image(gt_path)

        psnr_val = calculate_psnr(gen_img, gt_img)
        psnr_scores.append(psnr_val)

    round_avg = np.mean(psnr_scores)
    all_psnr_rounds.append(round_avg)
    print(f"🔁 Round {round_num}: Mean PSNR = {round_avg:.4f} dB")

final_avg = np.mean(all_psnr_rounds)
print("\n✅ Final Average PSNR over 20 rounds:", round(final_avg, 4), "dB")


🔁 Round 1: Mean PSNR = 5.0102 dB
🔁 Round 2: Mean PSNR = 5.0102 dB
🔁 Round 3: Mean PSNR = 5.0102 dB
🔁 Round 4: Mean PSNR = 5.0102 dB
🔁 Round 5: Mean PSNR = 5.0102 dB
🔁 Round 6: Mean PSNR = 5.0102 dB
🔁 Round 7: Mean PSNR = 5.0102 dB
🔁 Round 8: Mean PSNR = 5.0102 dB
🔁 Round 9: Mean PSNR = 5.0102 dB
🔁 Round 10: Mean PSNR = 5.0102 dB
🔁 Round 11: Mean PSNR = 5.0102 dB
🔁 Round 12: Mean PSNR = 5.0102 dB
🔁 Round 13: Mean PSNR = 5.0102 dB
🔁 Round 14: Mean PSNR = 5.0102 dB
🔁 Round 15: Mean PSNR = 5.0102 dB
🔁 Round 16: Mean PSNR = 5.0102 dB
🔁 Round 17: Mean PSNR = 5.0102 dB
🔁 Round 18: Mean PSNR = 5.0102 dB
🔁 Round 19: Mean PSNR = 5.0102 dB
🔁 Round 20: Mean PSNR = 5.0102 dB

✅ Final Average PSNR over 20 rounds: 5.0102 dB


In [15]:
pip install pytorch_msssim


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import numpy as np
from PIL import Image
import torch
from pytorch_msssim import ssim, ms_ssim

def load_image_tensor(path, size=(128, 128)):
    img = Image.open(path).convert("RGB")
    img = img.resize(size)
    img = np.array(img).astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)
    return img

generated_dir = "./SampledImgs/batch_1/"
groundtruth_dir = "./data/DIV2K_train_HR/"

custom_weights = [0.4, 0.3, 0.3]  # Only 3 levels
win_size = 7  # ✅ Keep this smaller to avoid assertion error

all_ssim_rounds = []
all_ms_ssim_rounds = []

for round_num in range(1, 21):
    ssim_scores = []
    ms_ssim_scores = []

    for i in range(1, 65):
        gen_path = os.path.join(generated_dir, f"sample_{i}.png")
        gt_path = os.path.join(groundtruth_dir, f"{str(i).zfill(4)}.png")

        if not os.path.exists(gen_path) or not os.path.exists(gt_path):
            continue

        gen_tensor = load_image_tensor(gen_path)
        gt_tensor = load_image_tensor(gt_path)

        ssim_val = ssim(gen_tensor, gt_tensor, data_range=1.0).item()
        ms_ssim_val = ms_ssim(
            gen_tensor, gt_tensor, data_range=1.0,
            weights=custom_weights,
            win_size=win_size  # ✅ Reduced window
        ).item()

        ssim_scores.append(ssim_val)
        ms_ssim_scores.append(ms_ssim_val)

    avg_ssim = np.mean(ssim_scores)
    avg_ms_ssim = np.mean(ms_ssim_scores)

    all_ssim_rounds.append(avg_ssim)
    all_ms_ssim_rounds.append(avg_ms_ssim)

    print(f"🔁 Round {round_num}: SSIM = {avg_ssim:.4f}, MS-SSIM = {avg_ms_ssim:.4f}")

final_ssim = np.mean(all_ssim_rounds)
final_ms_ssim = np.mean(all_ms_ssim_rounds)

print("\n✅ Final SSIM over 20 rounds:", round(final_ssim, 4))
print("✅ Final MS-SSIM over 20 rounds:", round(final_ms_ssim, 4))


🔁 Round 1: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 2: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 3: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 4: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 5: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 6: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 7: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 8: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 9: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 10: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 11: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 12: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 13: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 14: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 15: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 16: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 17: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 18: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 19: SSIM = 0.0029, MS-SSIM = 0.0120
🔁 Round 20: SSIM = 0.0029, MS-SSIM = 0.0120

✅ Final SSIM over 20 rounds: 0.0029
✅ Final MS-SSIM over 20 rounds: 0.012


In [22]:
import os
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np

# VGG Feature Extractor
class VGGFeatureExtractor(torch.nn.Module):
    def __init__(self, layers=['relu2_2'], device='cpu'):
        super(VGGFeatureExtractor, self).__init__()
        vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features.to(device).eval()
        self.layers = layers
        self.device = device
        self.layer_mapping = {
            'relu1_1': 0, 'relu1_2': 2,
            'relu2_1': 5, 'relu2_2': 7,
            'relu3_1': 10, 'relu3_2': 12, 'relu3_3': 14,
            'relu4_1': 17, 'relu4_2': 19, 'relu4_3': 21,
            'relu5_1': 24, 'relu5_2': 26, 'relu5_3': 28,
        }
        max_layer = max([self.layer_mapping[l] for l in layers])
        self.model = vgg[:max_layer + 1]

    def forward(self, x):
        features = {}
        for name, layer in self.model._modules.items():
            x = layer(x)
            for key, idx in self.layer_mapping.items():
                if int(name) == idx and key in self.layers:
                    features[key] = x
        return features

# Image loader
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def load_image_tensor(path, device='cpu'):
    img = Image.open(path).convert('RGB')
    img = preprocess(img).unsqueeze(0).to(device)
    return img

# Loop for perceptual similarity
device = 'cpu'
model = VGGFeatureExtractor(layers=['relu2_2'], device=device)
generated_dir = "./SampledImgs/batch_1/"
gt_dir = "./data/DIV2K_train_HR/"
image_indices = range(20)

l2_scores, cosine_scores = [], []

for i in image_indices:
    try:
        gen_path = os.path.join(generated_dir, f"sample_{i}.png")
        gt_path = os.path.join(gt_dir, sorted(os.listdir(gt_dir))[i])

        img1 = load_image_tensor(gen_path, device)
        img2 = load_image_tensor(gt_path, device)

        with torch.no_grad():
            f1 = model(img1)['relu2_2']
            f2 = model(img2)['relu2_2']

        l2 = F.mse_loss(f1, f2).item()
        cos_sim = F.cosine_similarity(f1.flatten(1), f2.flatten(1)).mean().item()

        l2_scores.append(l2)
        cosine_scores.append(cos_sim)

    except Exception as e:
        print(f"⚠️ Error comparing image {i}: {e}")

print("\n✅ Final Average Metrics over 20 samples:")
print(f"Perceptual Similarity (L2): {np.mean(l2_scores):.4f}")
print(f"Cosine Similarity: {np.mean(cosine_scores):.4f}")


⚠️ Error comparing image 0: [Errno 2] No such file or directory: './SampledImgs/batch_1/sample_0.png'

✅ Final Average Metrics over 20 samples:
Perceptual Similarity (L2): 184.1090
Cosine Similarity: 0.1579


In [30]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms
from scipy import linalg

device = "cuda" if torch.cuda.is_available() else "cpu"

# Preprocess for InceptionV3
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load InceptionV3 model
inception = inception_v3(weights=Inception_V3_Weights.DEFAULT, transform_input=False)
inception.fc = torch.nn.Identity()
inception.eval().to(device)

def get_activations(image_paths, model, batch_size=32):
    activations = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch = []
        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img = preprocess(img).unsqueeze(0)  # [1,3,299,299]
            batch.append(img)
        batch = torch.cat(batch).to(device)
        with torch.no_grad():
            act = model(batch)  # [B,2048]
        activations.append(act.cpu().numpy())
    return np.concatenate(activations, axis=0)

def calculate_fid(act1, act2):
    mu1, sigma1 = np.mean(act1, axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = np.mean(act2, axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2)
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean): covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2*covmean)
    return fid

# ==== Compute Average FID over 20 rounds ====
def compute_fid_avg(gen_dir, gt_dir, rounds=20):
    all_fid_scores = []
    gen_images = sorted([os.path.join(gen_dir, f) for f in os.listdir(gen_dir) if f.endswith('.png')])
    gt_images  = sorted([os.path.join(gt_dir, f) for f in os.listdir(gt_dir) if f.endswith('.png')])
    
    for i in range(rounds):
        sampled_gen = np.random.choice(gen_images, 50, replace=False)
        sampled_gt  = np.random.choice(gt_images, 50, replace=False)

        act_gen = get_activations(sampled_gen, inception)
        act_gt  = get_activations(sampled_gt, inception)

        fid = calculate_fid(act_gen, act_gt)
        all_fid_scores.append(fid)
        print(f"Round {i+1}: FID = {fid:.4f}")

    avg_fid = np.mean(all_fid_scores)
    print(f"\n✅ Average FID over {rounds} rounds = {avg_fid:.4f}")
    return avg_fid

# Example usage
compute_fid_avg("./SampledImgs/batch_1", "./data/DIV2K_train_HR", rounds=20)


Round 1: FID = 413.9255
Round 2: FID = 413.5248
Round 3: FID = 408.7175
Round 4: FID = 427.1364
Round 5: FID = 401.9757
Round 6: FID = 450.8836
Round 7: FID = 413.1036
Round 8: FID = 423.8680
Round 9: FID = 398.9120
Round 10: FID = 435.7685
Round 11: FID = 424.0186
Round 12: FID = 403.8796
Round 13: FID = 412.1881
Round 14: FID = 391.4142
Round 15: FID = 394.5985
Round 16: FID = 433.2858
Round 17: FID = 410.2757
Round 18: FID = 441.5389
Round 19: FID = 387.9187
Round 20: FID = 408.0925

✅ Average FID over 20 rounds = 414.7513


414.75131604970267

In [31]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# Preprocess for InceptionV3
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load InceptionV3 (output logits before softmax)
inception = inception_v3(weights=Inception_V3_Weights.DEFAULT, transform_input=False)
inception.fc = torch.nn.Identity()
inception.eval().to(device)

def get_predictions(image_paths, model, batch_size=32):
    preds = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch = []
        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img = preprocess(img).unsqueeze(0)  # [1, 3, 299, 299]
            batch.append(img)
        batch = torch.cat(batch).to(device)
        with torch.no_grad():
            logits = model(batch)
            softmax_preds = F.softmax(logits, dim=1)
        preds.append(softmax_preds.cpu().numpy())
    return np.concatenate(preds, axis=0)

def calculate_inception_score(preds, splits=10):
    N = preds.shape[0]
    scores = []
    for k in range(splits):
        part = preds[k * (N // splits): (k + 1) * (N // splits), :]
        py = np.mean(part, axis=0)
        kl_div = part * (np.log(part + 1e-10) - np.log(py + 1e-10))
        score = np.exp(np.mean(np.sum(kl_div, axis=1)))
        scores.append(score)
    return np.mean(scores), np.std(scores)

# ==== Run IS Evaluation ====
def compute_inception_score(image_folder, splits=10):
    image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')])
    preds = get_predictions(image_paths, inception)
    mean_is, std_is = calculate_inception_score(preds, splits=splits)
    print(f"\n✅ Inception Score: {mean_is:.4f} ± {std_is:.4f}")
    return mean_is, std_is

# Example usage
compute_inception_score("./SampledImgs/batch_1", splits=10)



✅ Inception Score: 1.0077 ± 0.0021


(1.0076773, 0.002106546)

In [33]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load InceptionV3
inception = inception_v3(weights=Inception_V3_Weights.DEFAULT, transform_input=False)
inception.fc = torch.nn.Identity()
inception.eval().to(device)

def get_predictions(image_paths, model, batch_size=32):
    preds = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch = []
        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img = preprocess(img).unsqueeze(0)
            batch.append(img)
        batch = torch.cat(batch).to(device)
        with torch.no_grad():
            logits = model(batch)
            softmax_preds = F.softmax(logits, dim=1)
        preds.append(softmax_preds.cpu().numpy())
    return np.concatenate(preds, axis=0)

def calculate_inception_score(preds, splits=10):
    N = preds.shape[0]
    scores = []
    for k in range(splits):
        part = preds[k * (N // splits): (k + 1) * (N // splits), :]
        py = np.mean(part, axis=0)
        kl_div = part * (np.log(part + 1e-10) - np.log(py + 1e-10))
        score = np.exp(np.mean(np.sum(kl_div, axis=1)))
        scores.append(score)
    return np.mean(scores), np.std(scores)

# ===== Safe Evaluation Over 20 Rounds =====
mean_list = []
std_list = []

for round_num in range(1, 21):
    image_folder = f"./SampledImgs/batch_{round_num}"
    
    if not os.path.exists(image_folder):
        print(f" Folder {image_folder} does not exist. Skipping...")
        continue
    
    image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')])
    
    if len(image_paths) == 0:
        print(f" No images in {image_folder}. Skipping...")
        continue

    preds = get_predictions(image_paths, inception)
    mean_is, std_is = calculate_inception_score(preds, splits=10)
    print(f"Round {round_num:02d} → Inception Score: {mean_is:.4f} ± {std_is:.4f}")
    mean_list.append(mean_is)
    std_list.append(std_is)

# ==== Final Summary ====
if mean_list:
    overall_mean = np.mean(mean_list)
    overall_std = np.mean(std_list)
    print(f"\n Final Avg Inception Score (over {len(mean_list)} rounds): {overall_mean:.4f} ± {overall_std:.4f}")
else:
    print(" No valid rounds processed.")


Round 01 → Inception Score: 1.0077 ± 0.0021
 Folder ./SampledImgs/batch_2 does not exist. Skipping...
 Folder ./SampledImgs/batch_3 does not exist. Skipping...
 Folder ./SampledImgs/batch_4 does not exist. Skipping...
 Folder ./SampledImgs/batch_5 does not exist. Skipping...
 Folder ./SampledImgs/batch_6 does not exist. Skipping...
 Folder ./SampledImgs/batch_7 does not exist. Skipping...
 Folder ./SampledImgs/batch_8 does not exist. Skipping...
 Folder ./SampledImgs/batch_9 does not exist. Skipping...
 Folder ./SampledImgs/batch_10 does not exist. Skipping...
 Folder ./SampledImgs/batch_11 does not exist. Skipping...
 Folder ./SampledImgs/batch_12 does not exist. Skipping...
 Folder ./SampledImgs/batch_13 does not exist. Skipping...
 Folder ./SampledImgs/batch_14 does not exist. Skipping...
 Folder ./SampledImgs/batch_15 does not exist. Skipping...
 Folder ./SampledImgs/batch_16 does not exist. Skipping...
 Folder ./SampledImgs/batch_17 does not exist. Skipping...
 Folder ./SampledImg

In [34]:
import os
import torch
import numpy as np
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms
from PIL import Image
from torch.nn.functional import adaptive_avg_pool2d
from scipy.linalg import sqrtm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Inception Model
inception = inception_v3(weights=Inception_V3_Weights.DEFAULT, transform_input=False)
inception.fc = torch.nn.Identity()
inception.eval().to(device)

def get_inception_features(image_paths, model, batch_size=32):
    feats = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch = []
        for path in batch_paths:
            img = Image.open(path).convert("RGB")
            img = preprocess(img).unsqueeze(0)
            batch.append(img)
        batch = torch.cat(batch).to(device)
        with torch.no_grad():
            act = model(batch)
            act = adaptive_avg_pool2d(act, (1, 1)).squeeze(-1).squeeze(-1)
        feats.append(act.cpu().numpy())
    return np.concatenate(feats, axis=0)

def polynomial_kernel(X, Y, degree=3, gamma=None, coef0=1):
    if gamma is None:
        gamma = 1.0 / X.shape[1]
    K = (gamma * X.dot(Y.T) + coef0) ** degree
    return K

def calculate_kid(X_feats, Y_feats, subsets=10, subset_size=1000):
    m = min(len(X_feats), len(Y_feats), subset_size)
    kid_scores = []
    for _ in range(subsets):
        x_idx = np.random.choice(len(X_feats), m, replace=False)
        y_idx = np.random.choice(len(Y_feats), m, replace=False)
        X = X_feats[x_idx]
        Y = Y_feats[y_idx]
        K_XX = polynomial_kernel(X, X)
        K_YY = polynomial_kernel(Y, Y)
        K_XY = polynomial_kernel(X, Y)
        m = float(m)
        kid = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1)) + \
              (np.sum(K_YY) - np.trace(K_YY)) / (m * (m - 1)) - \
              2 * np.sum(K_XY) / (m * m)
        kid_scores.append(kid)
    return np.mean(kid_scores), np.std(kid_scores)

# ===== KID Evaluation Over 20 Rounds =====
mean_kids = []
std_kids = []
gt_images = sorted([os.path.join("data/DIV2K_train_HR", f) for f in os.listdir("data/DIV2K_train_HR") if f.endswith(".png")])[:100]

gt_feats = get_inception_features(gt_images, inception)

for round_num in range(1, 21):
    image_folder = f"./SampledImgs/batch_{round_num}"
    if not os.path.exists(image_folder):
        print(f" {image_folder} missing. Skipping.")
        continue
    gen_images = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])
    if len(gen_images) == 0:
        print(f" No images in {image_folder}. Skipping.")
        continue

    gen_feats = get_inception_features(gen_images, inception)
    mean_kid, std_kid = calculate_kid(gen_feats, gt_feats, subsets=5)
    print(f"Round {round_num:02d} → KID: {mean_kid:.6f} ± {std_kid:.6f}")
    mean_kids.append(mean_kid)
    std_kids.append(std_kid)

# ==== Final KID Summary ====
if mean_kids:
    final_kid = np.mean(mean_kids)
    final_kid_std = np.mean(std_kids)
    print(f"\n Final Avg KID (20 rounds): {final_kid:.6f} ± {final_kid_std:.6f}")
else:
    print(" No valid rounds processed.")


ValueError: Input dimension should be at least 3

In [35]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models, transforms
from scipy import linalg

# Preprocessing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Inception Model (without classifier head)
inception = models.inception_v3(pretrained=True, transform_input=False)
inception.fc = nn.Identity()
inception.to(device)
inception.eval()

# Function to get features
def get_inception_features(image_paths, model, batch_size=32):
    feats = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch = []
        for path in batch_paths:
            img = Image.open(path).convert("RGB")
            img = preprocess(img).unsqueeze(0)
            batch.append(img)
        batch = torch.cat(batch).to(device)
        with torch.no_grad():
            act = model(batch)  # already [B, 2048]
        feats.append(act.cpu().numpy())
    return np.concatenate(feats, axis=0)

# Function to compute KID
def polynomial_mmd_averages(codes_g, codes_r, num_subsets=50, subset_size=1000, degree=3, gamma=None, coef0=1):
    m = subset_size
    n_g = codes_g.shape[0]
    n_r = codes_r.shape[0]
    mmds = []

    for _ in range(num_subsets):
        g = codes_g[np.random.choice(n_g, m, replace=True)]
        r = codes_r[np.random.choice(n_r, m, replace=True)]

        k_rr = (r @ r.T / r.shape[1]) if gamma is None else (gamma * r @ r.T + coef0) ** degree
        k_gg = (g @ g.T / g.shape[1]) if gamma is None else (gamma * g @ g.T + coef0) ** degree
        k_rg = (r @ g.T / r.shape[1]) if gamma is None else (gamma * r @ g.T + coef0) ** degree

        mmd = k_gg.mean() + k_rr.mean() - 2 * k_rg.mean()
        mmds.append(mmd)

    return np.mean(mmds), np.std(mmds)

# === Main Evaluation ===
mean_kids = []
std_kids = []

# Ground truth features (you can cache these if needed)
gt_dir = "data/DIV2K_train_HR"
gt_images = sorted([os.path.join(gt_dir, f) for f in os.listdir(gt_dir) if f.endswith(".png")])[:100]
gt_feats = get_inception_features(gt_images, inception)

for round_num in range(1, 21):
    image_folder = f"./SampledImgs/batch_{round_num}"
    if not os.path.isdir(image_folder):
        print(f" Folder not found: {image_folder}. Skipping...")
        continue

    gen_images = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])
    if len(gen_images) == 0:
        print(f" No images in {image_folder}. Skipping...")
        continue

    gen_feats = get_inception_features(gen_images, inception)

    mean_kid, std_kid = polynomial_mmd_averages(gen_feats, gt_feats)
    mean_kids.append(mean_kid)
    std_kids.append(std_kid)

    print(f" Round {round_num}: KID Mean = {mean_kid:.4f}, Std = {std_kid:.4f}")

# Summary
print("\n Final KID Scores over 20 rounds:")
print(f"Average KID Mean: {np.mean(mean_kids):.4f}")
print(f"Average KID Std : {np.mean(std_kids):.4f}")


 Round 1: KID Mean = 0.1066, Std = 0.0008
 Folder not found: ./SampledImgs/batch_2. Skipping...
 Folder not found: ./SampledImgs/batch_3. Skipping...
 Folder not found: ./SampledImgs/batch_4. Skipping...
 Folder not found: ./SampledImgs/batch_5. Skipping...
 Folder not found: ./SampledImgs/batch_6. Skipping...
 Folder not found: ./SampledImgs/batch_7. Skipping...
 Folder not found: ./SampledImgs/batch_8. Skipping...
 Folder not found: ./SampledImgs/batch_9. Skipping...
 Folder not found: ./SampledImgs/batch_10. Skipping...
 Folder not found: ./SampledImgs/batch_11. Skipping...
 Folder not found: ./SampledImgs/batch_12. Skipping...
 Folder not found: ./SampledImgs/batch_13. Skipping...
 Folder not found: ./SampledImgs/batch_14. Skipping...
 Folder not found: ./SampledImgs/batch_15. Skipping...
 Folder not found: ./SampledImgs/batch_16. Skipping...
 Folder not found: ./SampledImgs/batch_17. Skipping...
 Folder not found: ./SampledImgs/batch_18. Skipping...
 Folder not found: ./SampledImg

In [36]:
pip install lpips

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
import torch
import lpips
from PIL import Image
import numpy as np
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LPIPS model (AlexNet or VGG)
lpips_model = lpips.LPIPS(net='alex').to(device)  # Use 'vgg' or 'alex'

# Preprocess images to [-1, 1] for LPIPS
lpips_preprocess = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize for LPIPS model input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalize to [-1, 1]
])

# Function to load and prepare image
def load_lpips_image(path):
    img = Image.open(path).convert("RGB")
    img = lpips_preprocess(img).unsqueeze(0).to(device)
    return img

# LPIPS evaluation
lpips_scores = []

gt_images = sorted([os.path.join("data/DIV2K_train_HR", f) for f in os.listdir("data/DIV2K_train_HR") if f.endswith(".png")])[:64]

for round_num in range(1, 21):
    batch_folder = f"./SampledImgs/batch_{round_num}"
    if not os.path.isdir(batch_folder):
        print(f" Missing batch {round_num}, skipping.")
        continue

    gen_images = sorted([os.path.join(batch_folder, f) for f in os.listdir(batch_folder) if f.endswith(".png")])
    if len(gen_images) == 0:
        print(f" No images in batch {round_num}, skipping.")
        continue

    round_scores = []
    for gen_path, gt_path in zip(gen_images, gt_images):
        gen_tensor = load_lpips_image(gen_path)
        gt_tensor  = load_lpips_image(gt_path)

        with torch.no_grad():
            d = lpips_model(gen_tensor, gt_tensor).item()
        round_scores.append(d)

    avg_lpips = sum(round_scores) / len(round_scores)
    lpips_scores.append(avg_lpips)
    print(f" Round {round_num} - LPIPS: {avg_lpips:.4f}")

# Summary
print("\n Final LPIPS Summary over 20 rounds:")
print(f"Average LPIPS: {np.mean(lpips_scores):.4f}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /users/PFS0270/nikhilchatta/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 479MB/s] 


Loading model from: /users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


/users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, 

 Round 1 - LPIPS: 0.8485
 Missing batch 2, skipping.
 Missing batch 3, skipping.
 Missing batch 4, skipping.
 Missing batch 5, skipping.
 Missing batch 6, skipping.
 Missing batch 7, skipping.
 Missing batch 8, skipping.
 Missing batch 9, skipping.
 Missing batch 10, skipping.
 Missing batch 11, skipping.
 Missing batch 12, skipping.
 Missing batch 13, skipping.
 Missing batch 14, skipping.
 Missing batch 15, skipping.
 Missing batch 16, skipping.
 Missing batch 17, skipping.
 Missing batch 18, skipping.
 Missing batch 19, skipping.
 Missing batch 20, skipping.

 Final LPIPS Summary over 20 rounds:
Average LPIPS: 0.8485
